<a href="https://colab.research.google.com/github/pj747/qml-experiments/blob/main/wandbQNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -*- coding: utf-8 -*-
"""Hybrid.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1dfGZCYlB7p41Y9SOZS-oKcoCapN74N0h

###Required packages
"""


# !pip install pennylane-qulacs["gpu"] --upgrade
!pip install pennylane --upgrade
import pennylane as qml
from pennylane import qnn
import torch
from pennylane import numpy as np
from types import SimpleNamespace
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math

  Using cached https://files.pythonhosted.org/packages/61/ad/d18c5113d7c536c7b4544c2741795af7abde9557ef433c7814aa7d10d845/PennyLane-0.16.0-py3-none-any.whl


In [4]:
!pip install wandb
!wandb login


     |████████████████████████████████| 1.8MB 5.3MB/s 
     |████████████████████████████████| 102kB 8.2MB/s 
     |████████████████████████████████| 143kB 24.2MB/s 
     |████████████████████████████████| 174kB 21.4MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=3a530ccd9ee5b763427659d699817e6ecaa3ce7342aee894d0a8a04b43c8c0b7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=aafc1e6d3f895fab546336fd393d6da3149439129c866e09acc08586904a9b95
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
def wandbSweep(config):

    """###Data Preparation
    The Wisconsin Breast Cancer dataset is prepared for two prediction formats - 
    a list of one-hot vectors for a classical neural network, and a list of label predictions for the quantum case.
    """

    dataSet = load_breast_cancer()
    X = dataSet.data
    Y = dataSet.target
    if config.standardScaling:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    Y_label = Y * 2 - np.ones(len(Y))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y_label, test_size=0.1, random_state=1)

    Y_01_train = torch.as_tensor(((Y_train + np.ones(len(Y_train)))//2)).to(torch.int64)
    Y_hot_train = torch.nn.functional.one_hot(Y_01_train, num_classes=2)

    Y_01_test = torch.as_tensor(((Y_test + np.ones(len(Y_test)))//2)).to(torch.int64)
    Y_hot_test = torch.nn.functional.one_hot(Y_01_test, num_classes=2)

    """### Quantum circuit creation
    This cell sets up a quantum circuit with the appropriate configuration
    """

    numQubits = config.numQubits
    dev = qml.device("default.qubit", wires=numQubits)
    @qml.qnode(dev)
    def qnode(inputs, weights):
        for i in range(config.numLayers):
            if config.vectorNorm == "Yes":
                norm = np.linalg.norm(inputs.clone().detach())
                norm = norm if norm !=0 else 1
            else:
                norm = 2 * math.pi
            for k in range(0, len(inputs)-numQubits, numQubits):
                for j in range(numQubits):
                    qml.RX(inputs[k+j]*2*math.pi/norm, wires=j)
            for j in range(numQubits):
                qml.Rot(weights[j][i][0], weights[j][i][1], weights[j][i][2], wires = [j])
            if config.fullEntangle == "Yes":
                for j in range(numQubits):
                    for i in range(j):
                        qml.CZ(wires=[j,i])
            else:
                for j in range(numQubits-1):
                    qml.CZ(wires=[j,j+1])
            
            ##qml.Rot(*weights[0], wires=[0])
        if config.end == "quantum":   
            return qml.expval(qml.PauliZ(0))
        else:
            return [qml.expval(qml.PauliZ(wires=i)) for i in range(numQubits)]

    weight_shapes = {"weights" : (config.numQubits, config.numLayers, 3)}
    qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)


    """### Model creation
    This cell instantiates the actual model to be run
    """

    if config.hiddenLayer != 0:
        clayer_1 = torch.nn.Linear(30, config.hiddenLayer)
        
        if config.end == "quantum":
            layers = [clayer_1, qlayer]
        else:
            clayer_2 = torch.nn.Linear(config.numQubits, 2)
            softmax = torch.nn.Softmax(dim=1)
            if config.start == "classical":
                layers = [clayer_1, qlayer, clayer_2, softmax]
            else:
                layers = [qlayer, clayer_2, softmax]
    else:
        layers = [qlayer]

    torch.nn.init.uniform_(qlayer.weights, a=0.0, b=0.001)
    model = torch.nn.Sequential(*layers)

    """### Training"""

    opt = torch.optim.Adam(model.parameters(), lr=0.01)
    loss = torch.nn.MSELoss()
    x_train = torch.tensor(X_train, requires_grad=True).float()

    y_train = torch.tensor(Y_train, requires_grad=False).float() if config.end == "quantum" else Y_hot_train.float()


    batch_size = config.batchSize
    batches = y_train.shape[0]/batch_size // batch_size

    data_loader = torch.utils.data.DataLoader(
        list(zip(x_train, y_train)), batch_size=5, shuffle=True, drop_last=True
    )

    epochs = config.epochs

    for epoch in range(epochs):

        running_loss = 0

        for xs, ys in data_loader:
            opt.zero_grad()

            loss_evaluated = loss(model(xs), ys)
            loss_evaluated.backward()

            opt.step()

            running_loss += loss_evaluated

        avg_loss = running_loss / batches
        print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))
        wandb.log({"train_loss":avg_loss})

    """### Training set accuracy"""

    y_pred = model(x_train)
    y_pred = y_pred.detach().numpy()  
    if config.end == "quantum":
        threshold = lambda x: 1 if x > 0 else -1 
        vfunc = np.vectorize(threshold)
        y_pred = vfunc(y_pred)
        actual = Y_train
    else:
        y_pred = np.argmax(y_pred, axis=1)
        actual = Y_01_train.detach().numpy()

    correct = [1 if p == p_true else 0 for p, p_true in zip(y_pred, actual)]
    accuracy = sum(correct) / len(correct)
    print(f"Accuracy: {accuracy * 100}%")
    wandb.log({"train_acc":accuracy})


    """### Testing set accuracy"""

    x_test = torch.tensor(X_test, requires_grad=True).float()
    y_pred = model(x_test)
    y_pred = y_pred.detach().numpy()  
    if config.end == "quantum":
        threshold = lambda x: 1 if x > 0 else -1 
        vfunc = np.vectorize(threshold)
        y_pred = vfunc(y_pred)
        actual = Y_test
    else:
        y_pred = np.argmax(y_pred, axis=1)
        actual = Y_01_test.detach().numpy()

    correct = [1 if p == p_true else 0 for p, p_true in zip(y_pred, actual)]
    accuracy = sum(correct) / len(correct)
    wandb.log({"test_acc":accuracy})
    print(f"Accuracy: {accuracy * 100}%")
    wandb.finish()

In [ ]:

"""###Global config
For the rest of the notebook, config needs to be defined here.
"""
import wandb

configDefault = SimpleNamespace(
    standardScaling = "Yes",
    numQubits = 2,
    vectorNorm = "Yes",
    numLayers = 1,
    end = "classical",
    fullEntangle = "Yes",
    epochs = 6,
    hiddenLayer = 5,
    start = "quantum",
    batchSize = 5
)

wandb.init(config = configDefault, project='hybrid-qnn')
config = wandb.config
wandbSweep(config)
wandb.finish()

Average loss over epoch 1: 1.2364
Average loss over epoch 2: 1.1822
Average loss over epoch 3: 1.1840


In [ ]:
wandb.agent('1fny5wr2', function=wandbSweep, entity = 'pj747', project='hybrid-qnn')